<a href="https://colab.research.google.com/github/Saira1302/Saira1302/blob/AI/fine_tune_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets evaluate rouge_score accelerate

In [ ]:
import transformers

transformers.__version__

'4.44.2'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetInfo(id='amirveyseh/acronym_identification', author='amirveyseh', sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=925, downloads_all_time=None, likes=19, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'source_datasets:original', 'language:en', 'license:mit', 'size_categories:10K<n<100K', 'format:parquet', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'arxiv:2010.14678', 'region:us', 'acronym-identification'], card_data=None, siblings=None)
DatasetInfo(id='ade-benchmark-corpus/ade_corpus_v2', author='ade-benchmark-corpus', sha='4ba01c71687dd7c996597042449448ea312126cf', 

In [ ]:
from datasets import load_dataset

cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0")

cnn_news_summary_ds

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:

cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0", split = "train[:3%]")

cnn_news_summary_ds

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 8613
})

In [ ]:
cnn_news_summary_ds.shape

(8613, 3)

In [ ]:

print(cnn_news_summary_ds.description)

In [ ]:

cnn_news_summary_ds.features

{'article': Value(dtype='string', id=None),
 'highlights': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [ ]:

cnn_news_summary_ds.homepage

''

In [ ]:
cnn_news_summary_ds[0]


{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:

cnn_news_summary_ds = cnn_news_summary_ds.train_test_split(test_size = 0.2, seed = 42)

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

In [ ]:
def clean_txt(example):
    for txt in ["article", "highlights"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example

In [ ]:
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_txt)

cleaned_cnn_news_summary_ds

Map:   0%|          | 0/6890 [00:00<?, ? examples/s]

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

In [ ]:
cnn_news_summary_ds["train"]["article"][7]

'(CNN)  -- Four-time defending champion Rafael Nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to Sweden\'s Robin Soderling at the French Open on Sunday. Soderling savors his unlikely triumph against Nadal in the French Open fourth round. Soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at Roland Garros and was a short-priced favorite to win a record fifth straight title. But Soderling never looked back after breezing through the first set of their fourth round match and closed out victory as Nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. The Spaniard tried to look on the positives after his shock defeat. "It\'s not a tragedy, losing here in Paris," he told the official tournament Web site www.rolandgarros.com . "It had to happen one day, and this is an excellent season for me. "Of course it\'s a bit sad, but I have to overcome this as quickly as possible. No one remembers defea

In [ ]:

cnn_news_summary_ds["train"]["highlights"][0]

'Maradona staying a clinic in Italy when tax authorities swooped .\nReports: Argentina coach owes millions in taxes from his time playing in Italy .\nMaradona says Serie A club should have paid the taxes .'

In [ ]:
cleaned_cnn_news_summary_ds["train"]["highlights"][0]

'maradona staying a clinic in italy when tax authorities swooped .reports: argentina coach owes millions in taxes from his time playing in italy .maradona says serie a club should have paid the taxes .'

In [ ]:
EXAMPLE_TEXT_INDEX = 33

example_text = cleaned_cnn_news_summary_ds["test"]["article"][EXAMPLE_TEXT_INDEX]

example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

In [ ]:

MODEL_NAME = "t5-small"

In [ ]:

# Use a pipeline as a high-level helper
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)
summary_txt = summarizer(prefix + example_text)
summary_txt


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever . lancey, a 10-week-old puppy, was hand-delivered to them earlier this week . he's the most human of any dog we've ever had, the company says ."}]

In [ ]:

summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)

summary_txt = summarizer(prefix + example_text)
summary_txt

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': 'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'}]

In [ ]:
ref_txt = cleaned_cnn_news_summary_ds["test"]["highlights"][EXAMPLE_TEXT_INDEX]
ref_txt

"couple won auction to clone family dog, biotech company says .lancey is world's first commercially cloned dog, company says .dna of deceased dog sent to s. korea, and cloned puppy born november 18 .humane society says it's against commercial cloning of animals ."

In [ ]:
import evaluate

rouge = evaluate.load("rouge")
rouge


EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"
ex_2 = "the big elephant was found near the river and made everyone glad"


In [ ]:

results = rouge.compute(predictions = [ex_1], references = [reference])
results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

In [ ]:
results = rouge.compute(predictions = [ex_2], references = [reference])
results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

In [ ]:

summary_txt[0]["summary_text"]

'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'

In [ ]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.13157894736842105,
 'rouge2': 0.027027027027027025,
 'rougeL': 0.07894736842105263,
 'rougeLsum': 0.07894736842105263}

In [ ]:
article_texts = cleaned_cnn_news_summary_ds["test"]["article"]
article_summaries = cleaned_cnn_news_summary_ds["test"]["highlights"]

In [ ]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [03:14<00:00,  3.88s/it]


In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.3257425996249486,
 'rouge2': 0.13981650875711288,
 'rougeL': 0.23995664224058197,
 'rougeLsum': 0.24037684392848543}

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.34146341463414637,
  0.38805970149253727,
  0.2608695652173913,
  0.28125,
  0.27848101265822783,
  0.18181818181818182,
  0.3855421686746988,
  0.24615384615384614,
  0.5974025974025975,
  0.3076923076923077,
  0.33766233766233766,
  0.3235294117647059,
  0.17821782178217824,
  0.1917808219178082,
  0.2278481012658228,
  0.3582089552238805,
  0.2702702702702703,
  0.3235294117647059,
  0.2682926829268293,
  0.32432432432432434,
  0.35443037974683544,
  0.28205128205128205,
  0.40540540540540543,
  0.10666666666666667,
  0.3116883116883117,
  0.36363636363636365,
  0.32558139534883723,
  0.5757575757575758,
  0.42857142857142855,
  0.24444444444444444,
  0.271604938271605,
  0.37288135593220334,
  0.3,
  0.13157894736842105,
  0.3820224719101123,
  0.44,
  0.4507042253521127,
  0.5217391304347826,
  0.647887323943662,
  0.2105263157894737,
  0.1038961038961039,
  0.2153846153846154,
  0.1590909090909091,
  0.3711340206185567,
  0.3548387096774194,
  0.2571428571428571,
  

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(38, 12)

In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,firefighter jimmy senk went inside a burning a...,horia cretan became a hero for carrying boy fr...
1,nigeria's princess nikky onyeri has dedicated ...,health advocate from nigeria campaigns for bet...
2,troy davis has always maintained his innocence...,supreme court's latest ruling means troy davis...
3,"more than 11,500 public servants have been sus...",international transparency poll: businesses in...
4,human rights watch says the displaced sri lank...,"group: more than 280,000 displaced tamil civil..."


In [ ]:

print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(38, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(38, "Reference_summaries"))

Actual Summary
the statue of kate moss will be displayed in the british museum in october . it is the largest gold statue built since ancient egypt . the statue will be part of a group of major sculptures .
Reference Summary
statue of kate moss billed as largest gold statue built since ancient egypt .called siren, the statue will be on display at the british museum .moss statue will be displayed in gallery housing ancient greek sculpture .


In [ ]:

print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(12, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(12, "Reference_summaries"))

Actual Summary
mobile computer chips have gotten so powerful and affordable, and it doesn't make much sense to ship a gadget with dedicated, stagnant functionality . developers have created 100,000 apps for the store at no cost to apple, which takes a cut of revenues .
Reference Summary
companies are opening app stores and bringing applications to tvs, netbooks and even cars .intel launched an app store for netbooks at the consumer electronics show this week .samsung will launch an app store later this year for hdtvs and blu-ray players .ford motor co. will make select apps available for new vehicles later in 2010 .


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
tokenizer(["four-time defending champion"])

{'input_ids': [[662, 18, 715, 3, 20309, 6336, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:

tokens = tokenizer.tokenize("four-time defending champion")
tokens


['▁four', '-', 'time', '▁', 'defending', '▁champion']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids


[662, 18, 715, 3, 20309, 6336]

In [ ]:

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["highlights"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:

tokenized_cnn_news_summary_ds = cleaned_cnn_news_summary_ds.map(preprocess_function, batched = True)
tokenized_cnn_news_summary_ds

Map:   0%|          | 0/6890 [00:00<?, ? examples/s]

In [ ]:

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "cnn_news_summary_model_trained_on_reduced_data",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_cnn_news_summary_ds["train"],
    eval_dataset = tokenized_cnn_news_summary_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,1.624171,0.217300,0.093200,0.182600,0.182700,19.000000
2,1.920300,1.607420,0.216700,0.093300,0.182600,0.182600,19.000000
3,1.822100,1.604041,0.217800,0.093900,0.184000,0.183900,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1293, training_loss=1.8592027043153037, metrics={'train_runtime': 1421.2787, 'train_samples_per_second': 14.543, 'train_steps_per_second': 0.91, 'total_flos': 5595030072852480.0, 'train_loss': 1.8592027043153037, 'epoch': 3.0})

In [ ]:

trainer.push_to_hub()

events.out.tfevents.1727286990.c515916332ef.4209.0:   0%|          | 0.00/8.35k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Themarina/cnn_news_summary_model_trained_on_reduced_data/commit/1d501cc6d025f4e9e6c33754ac37480667956f0b', commit_message='End of training', commit_description='', oid='1d501cc6d025f4e9e6c33754ac37480667956f0b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:

example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

In [ ]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline(
    "summarization",
    model = "jamil226/cnn_news_summary_model_trained_on_reduced_data",
    truncation=True
)
summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': 'edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever .the 10-week-old puppy, dubbed lancey, was hand-delivered to them earlier this week .otos were one of five families to bid and win a bioarts auction for a chance .dna sample was sent to the sooam biotech research foundation in seoul .'}]